# Exporting MXNet models to ONNX

In this tutorial, we will show how you can save MXNet models to ONNX format.
ONNX exporter it a part of [MXNet repository](https://github.com/apache/incubator-mxnet/tree/master/python/mxnet/contrib/onnx/mx2onnx).

Current MXNet-ONNX import and export operator support and coverage can be found [here](Operator support and coverage - https://cwiki.apache.org/confluence/display/MXNET/ONNX):


## Step 1: Installations:
First, install ONNX 1.2.1 version. Follow instructions on [ONNX repo](https://github.com/onnx/onnx).

Make sure to install latest MXNet either from source or pip.

```bash
pip install mxnet --pre
```

* Note: ONNX exporter will be released as a part of MXNet v1.3. 

## Step 2: Prepare MXNet model to convert to ONNX:

Let's try out pretrained resnet model from [MXNet model zoo](http://data.mxnet.io/models/).

In [1]:
import mxnet as mx
import numpy as np
from mxnet.contrib import onnx as onnx_mxnet

In [2]:
# Download pretrained resnet model - json and params from mxnet model zoo.
path='http://data.mxnet.io/models/imagenet/'
[mx.test_utils.download(path+'resnet/18-layers/resnet-18-0000.params'),
 mx.test_utils.download(path+'resnet/18-layers/resnet-18-symbol.json'),
 mx.test_utils.download(path+'synset.txt')]

['resnet-18-0000.params', 'resnet-18-symbol.json', 'synset.txt']

## Step 3: Use MXNet to ONNX exporter:

MXNet's ONNX "export_model" API accepts following inputs: 


In [3]:
help(onnx_mxnet.export_model)

Help on function export_model in module mxnet.contrib.onnx.mx2onnx.export_model:

export_model(sym, params, input_shape, input_type=<type 'numpy.float32'>, onnx_file_path=u'model.onnx', verbose=False)
    Exports the MXNet model file, passed as a parameter, into ONNX model.
    Accepts both symbol,parameter objects as well as json and params filepaths as input.
    Operator support and coverage - https://cwiki.apache.org/confluence/display/MXNET/ONNX
    
    Parameters
    ----------
    sym : str or symbol object
        Path to the json file or Symbol object
    params : str or symbol object
        Path to the params file or params dictionary. (Including both arg_params and aux_params)
    input_shape : List of tuple
        Input shape of the model e.g [(1,3,224,224)]
    input_type : data type
        Input data type e.g. np.float32
    onnx_file_path : str
        Path where to save the generated onnx file
    verbose : Boolean
        If true will print logs of the model conver

From the API description, you can see that `export_model` API accepts 2 kinds of inputs:

### MXNet sym, params objects:

    This is useful if you are training a model. At the end of training, you just need to invoke the `export_model` function and provide sym and params objects as inputs with other attributes to save the model in ONNX format.
    
### MXNet's exported json and params files:

    This is useful if you have pretrained models and you want to convert them to ONNX format.

In this tutorial, we will show second usecase: 

In [4]:
# Downloaded input symbol and params files
sym = 'resnet-18-symbol.json'
params = 'resnet-18-0000.params'
# Standard Imagenet input - 3 channels, 224*224
input_shape = (1,3,224,224)
# Path of the output file
onnx_file = 'mxnet_exported_resnet50.onnx'

In [5]:
# Invoke export model API. It returns path of the converted onnx model
converted_model_path = onnx_mxnet.export_model(sym, params, [input_shape], np.float32, onnx_file)

In [6]:
print(converted_model_path)

mxnet_exported_resnet50.onnx


## Step 4: Check validity

You can check validity of the converted ONNX model by using ONNX checker tool as follows:

In [7]:
from onnx import checker
import onnx
# Load onnx model
model_proto = onnx.load(converted_model_path)

# Check if converted ONNX protobuf is valid
checker.check_graph(model_proto.graph)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model_proto.graph))

graph mxnet_converted_model (
  %data[FLOAT, 1x3x224x224]
) initializers (
  %bn_data_gamma[FLOAT, 3]
  %bn_data_beta[FLOAT, 3]
  %bn_data_moving_mean[FLOAT, 3]
  %bn_data_moving_var[FLOAT, 3]
  %conv0_weight[FLOAT, 64x3x7x7]
  %bn0_gamma[FLOAT, 64]
  %bn0_beta[FLOAT, 64]
  %bn0_moving_mean[FLOAT, 64]
  %bn0_moving_var[FLOAT, 64]
  %stage1_unit1_bn1_gamma[FLOAT, 64]
  %stage1_unit1_bn1_beta[FLOAT, 64]
  %stage1_unit1_bn1_moving_mean[FLOAT, 64]
  %stage1_unit1_bn1_moving_var[FLOAT, 64]
  %stage1_unit1_conv1_weight[FLOAT, 64x64x3x3]
  %stage1_unit1_bn2_gamma[FLOAT, 64]
  %stage1_unit1_bn2_beta[FLOAT, 64]
  %stage1_unit1_bn2_moving_mean[FLOAT, 64]
  %stage1_unit1_bn2_moving_var[FLOAT, 64]
  %stage1_unit1_conv2_weight[FLOAT, 64x64x3x3]
  %stage1_unit1_sc_weight[FLOAT, 64x64x1x1]
  %stage1_unit2_bn1_gamma[FLOAT, 64]
  %stage1_unit2_bn1_beta[FLOAT, 64]
  %stage1_unit2_bn1_moving_mean[FLOAT, 64]
  %stage1_unit2_bn1_moving_var[FLOAT, 64]
  %stage1_unit2_conv1_weight[FLOAT, 64x64x3x3]
  %stage1

## What's next

Take a look at [other tutorials, including importing of ONNX models to other frameworks](https://github.com/onnx/tutorials/tree/master/tutorials)